In [35]:
import csv
import pandas as pd
import psycopg2

In [36]:
#Приведение CSV файла к корректному формату
movieId = []
movieName = []
movieYear = []
with open('IMDBMovieTitles.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        try:
            mn, not_good_y = row[1].split('(')
            y, bad_y = not_good_y.split(')')
        except:
            continue
 
        movieId.append(row[0])
        movieName.append(mn.strip())
        movieYear.append(y)

In [4]:
#Сохранение правильного CSV файла
df = pd.DataFrame({'id':   movieId,
                   'year': movieYear, 
                   'name': movieName})
df.to_csv('newCSV.csv', index=False)

In [37]:
#Connection
conn = psycopg2.connect(database='lab1', user='postgres',
    password='12345', host='localhost')#, port=5432)

In [38]:
#Создание новой таблицы в Postgres
'''''''''
cur = conn.cursor()
cur.execute("create table movies (id integer generated by default as " + 
            "identity primary key, year integer, name text)")

conn.commit()
cur.close()
'''''''''

'\ncur = conn.cursor()\ncur.execute("create table movies (id integer generated by default as " + \n            "identity primary key, year integer, name text)")\n\nconn.commit()\ncur.close()\n'

In [42]:
#Функция поиска
def find(text, connection):
    txt = text.split()
    year = ''
    
    #Поиск года в запросе
    for word in txt:
        if word.isnumeric() and len(word) == 4:
            try:
                if int(word) >= 1894:
                    year = word
                    break
            except:
                continue
                
    #Если год есть, то удалем его из запроса
    if year != '':
        text = text.replace(year, '', 1)
    
    #Странное изменение строки для того, чтобы работали SQL запросы
    text = text.strip()
    text = text.replace(' ', '% %')
    text = '%' + text + '%'
    
    #Создание курсора
    cur = connection.cursor()
    
    #SQL запросы
    ret = []

    if text == '%%' and year == '':
        newText = str(input('Введите поисковой запрос: '))
        return find(newText, connection)
    elif year == '':
        cur.execute('Select * from public.movies where name like %(название фильма)s limit 10', 
                    {'название фильма': text})
    else:
        cur.execute('Select * from movies where name like %(название фильма)s and year = %(дата выхода)s limit 10',
                   {'название фильма': text, 'дата выхода': year})
        
    results = cur.fetchall()
    
    #Закрытие курсора
    cur.close()
    
    for res in results:
        ret.append('Название фильма: ' + res[2] +
                   ' Дата выхода: '    + str(res[1]) +
                   ' Id фильма: '      + str(res[0]))
    
    if len(ret) == 0:
        ret.append('Ничего не найдено.')
        
    return ret

In [45]:
#Обёртка поиска
def find_smth():
    searchQuery = str(input('Введите поисковой запрос: '))
    results = find(searchQuery, conn)
    for result in results:
        print(result)

Поисковые запросы

In [47]:
#По одному из слов названия фильма
find_smth()

Введите поисковой запрос: Class
Название фильма: A Touch of Class Дата выхода: 1973 Id фильма: 70819
Название фильма: Class of 1984 Дата выхода: 1982 Id фильма: 83739
Название фильма: Class of Nuke Em High Дата выхода: 1986 Id фильма: 90849
Название фильма: Cutting Class Дата выхода: 1989 Id фильма: 97136
Название фильма: Class of 1999 Дата выхода: 1990 Id фильма: 99277
Название фильма: Class Дата выхода: 1991 Id фильма: 101590
Название фильма: Class Act Дата выхода: 1992 Id фильма: 103978
Название фильма: X-Men First Class Дата выхода: 2011 Id фильма: 1270798
Название фильма: The Class of 92 Дата выхода: 2013 Id фильма: 3311384


In [48]:
#По всем словам названия
find_smth()

Введите поисковой запрос: A Touch of Class
Название фильма: A Touch of Class Дата выхода: 1973 Id фильма: 70819


In [49]:
#По частичным словам из названия фильма
find_smth()

Введите поисковой запрос: ouch ass
Название фильма: A Touch of Class Дата выхода: 1973 Id фильма: 70819


In [50]:
#По году и названию (части названия)
find_smth()

Введите поисковой запрос: ass 1973
Название фильма: A Touch of Class Дата выхода: 1973 Id фильма: 70819


In [51]:
#По году
find_smth()

Введите поисковой запрос: 2012
Название фильма: Foodfight Дата выхода: 2012 Id фильма: 249516
Название фильма: On the Road Дата выхода: 2012 Id фильма: 337692
Название фильма: The Three Stooges Дата выхода: 2012 Id фильма: 383010
Название фильма: John Carter Дата выхода: 2012 Id фильма: 401729
Название фильма: Gambit Дата выхода: 2012 Id фильма: 404978
Название фильма: The Possession Дата выхода: 2012 Id фильма: 431021
Название фильма: Lincoln Дата выхода: 2012 Id фильма: 443272
Название фильма: Os Penetras Дата выхода: 2012 Id фильма: 452664
Название фильма: Life of Pi Дата выхода: 2012 Id фильма: 454876
Название фильма: Being Flynn Дата выхода: 2012 Id фильма: 455323
